# DNBLab Jupyter Notebook Skript 

## Abfrage der SRU-Schnittstelle



### Importieren passender Bibliotheken  <a class="anchor" id="Teil1"></a>

In [1]:
import requests
import pandas as pd
import lxml.etree as ET
from bs4 import BeautifulSoup as soup
import unicodedata

### Schnelle Abfrage (100 erste Treffer, nur Rückgabe der Treffermenge)  <a class="anchor" id="Teil1"></a>

In [4]:
def sru_dnb_simple(query): 

    dnb_url = "https://services.dnb.de/sru/dnb"
    parameter = {'version' : '1.1' , 'operation' : 'searchRetrieve' , 'query' : query,
                 'recordSchema' : 'MARC21-xml', 'maximumRecords': '100'} 

    r = requests.get(dnb_url, params = parameter)
    response = soup(r.content)
    number = response.find_all('numberofrecords')[0].text
    
    return number
    
    

In [8]:
#myquery1 = sru_dnb_simple('HSS=diss* and (sgt="3*" or sgt="5*" or sgt="6*") and jhr="1980"')
myquery1 = sru_dnb_simple('HSS=diss* and (sgt="5*") and jhr="1980"')
print(myquery1, "Ergebnisse")

3838 Ergebnisse


### Abfrage aller Ergebnisse über SRU - ACHTUNG: Das kann dauern! 

Bei sehr umfangreichen Anfragen kann es ggf. zu Abbrüchen kommen - in diesem Fall bietet es sich an, die Abfrage entweder noch einmal aufzuteilen oder bspw. nach je 100 Schliefendurchläufen eine Pause einzubauen. 

In [9]:
def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    r = requests.get(base_url, params=params)
    xml = soup(r.content)
    records = xml.find_all('record', {'type':'Bibliographic'})
    
    if len(records) < 100:
        
        return records
    
    else:
        
        num_results = 100
        i = 101
        while num_results == 100:
            
            params.update({'startRecord': i})
            r = requests.get(base_url, params=params)
            xml = soup(r.content)
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=100
            num_results = len(new_records)
            
        return records



In [17]:
myquery = dnb_sru('HSS=diss* and (sgt="5*") and jhr="1980"')
print(len(myquery), "Ergebnisse")

3838 Ergebnisse


In [16]:
# Ausgabe EINES Treffers aus der XML-Antwort der Schnittstelle 
# Die Zahl in Klammern gibt die Nummer des angezeigten Treffers wieder, beginnend bei 0 für den ersten Treffer. 

#print(myquery[0])

### Verarbeiten der Ergebnisse  <a class="anchor" id="Teil4"></a>

Hier wird die XML-Antwort der SRU-Schnittstelle Treffer für Treffer übergeben und bestimmte Inhalte herausgezogen, um im Anschluss in ein Dataframe zur weiteren Arbeit und Analyse übertragen zu werden.

In [18]:
def parse_record(record):
    
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = ET.fromstring(unicodedata.normalize("NFC", str(record)))
    
    #idn
    idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'
    
    # titel
    titel = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        titel = titel[0].text
    except:
        titel = "unknown"
        
    
    # author
    author = xml.xpath("marc:datafield[@tag = '100']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        author = author[0].text
    except:
        author = "unknown"
        
        
    #gnd
    gnd = xml.xpath("marc:datafield[@tag = '100']/marc:subfield[@code = '0']", namespaces=ns)
    try:
        gnd = gnd[0].text
    except:
        gnd = "none"
  

    # supervisor (maybe):
    superv = xml.xpath("marc:datafield[@tag = '700']/marc:subfield[@code = 'a']", namespaces=ns)
    superrole = xml.xpath("marc:datafield[@tag = '700']/marc:subfield[@code = 'e']", namespaces=ns)
    try:
        superv = superv[0].text
        superrole = superrole[0].text
        supervisor = superv + " - " + superrole
    except:
        supervisor = "unknown"
    
    
    meta_dict = {"idn":idn, "titel":titel, "author":author, "gnd":gnd, "supervisor":supervisor}
    
    return meta_dict



In [19]:
output = [parse_record(record) for record in myquery]
df = pd.DataFrame(output)
df

,idn,titel,author,gnd,supervisor
0,801327865,1-O-Hydroxicinnamyl-β-D-glucopyranosen [OHyd...,"Reichert, Angelika",none,unknown
1,810017954,"1,1-Di-tert-butyl-1-silacyclobutan [Ditertbu...","Rankers, Reinhard",(DE-588)1042386145,unknown
2,810642514,"1,3-Benzazaphosphole [Benzazaphosphole] und ...","Vollmer, Gerd-Rainer",(DE-588)1234420597,unknown
3,810622653,"1,3-Dipolare [Dipolare] Cycloadditionen arom...","Temme, Robert",none,unknown
4,810830981,13C-13C-Kopplungskonstanten [C-C-Kopplungsko...,"Wardeiner, Jürgen",none,unknown
...,...,...,...,...,...
3833,820035548,Zweitsinn,"Kandler, Günther",none,unknown
3834,810635585,Zytochrom P-450 [P] in der Hefe Saccharomyce...,"Kuhl, Rosemarie",none,unknown
3835,211861316,Zytofluorometrische Untersuchung DNA-spezifisc...,"Hauser-Urfer, Ingrid Helen",none,unknown
3836,211861154,Zytofluorometrische Untersuchungen zur Charakt...,"Müller, Beat Walter",none,unknown


In [115]:
#Speichern der Ergebnisse aus dem Dataframe als CSV-Datei: 
df.to_csv("Abfrage.csv", index=False) 

In [122]:
#Speichern der Ergebnisse aus dem Dataframe als Excel-Datei: 
df.to_excel("Abfrage.xlsx", index=False, encoding='utf8') 

### Filtern des Dataframe

In [20]:
## Show all items where no author was found: 

noauthors = df.query('author == "unknown"')
noauthors

,idn,titel,author,gnd,supervisor
263,810363992,"Beiträge zur ""Energetischen Geomorphologie"" in...",unknown,none,"Hempel, Ludwig - Verfasser"
408,208938184,Biostratigraphie und Ammonitenfauna des Unter-...,unknown,none,unknown
695,830662197,Deutsche Soldatenlieder,unknown,none,unknown
2156,213230720,Primärstruktur des Polypeptids IV aus Cytochro...,unknown,none,unknown
3577,800527038,Wirkungsanalyse im Rahmen der Landschaftsplanung,unknown,none,"Krause, Christian L. - Mitwirkender"
